실행환경 : 구글 코랩 pro  
사용 딥러닝 프레임워크 : mmaction2 (https://github.com/open-mmlab/mmaction2)  
사용한 모델 : VideoSwin  

In [ ]:
import numpy as np
import pandas as pd
from operator import itemgetter

In [ ]:
# install dependencies: (if your colab has CUDA 11.8)
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
# install MMEngine, MMCV and MMDetection using MIM
%pip install -U openmim
!mim install mmengine
!mim install "mmcv>=2.0.0"

# Install mmaction2
!rm -rf mmaction2
!git clone https://github.com/open-mmlab/mmaction2.git -b main
%cd mmaction2

!pip install -e .

# Install some optional requirements
!pip install -r requirements/optional.txt

Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.1.0/index.html
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.1.0/index.html
Cloning into 'mmaction2'...
remote: Enumerating objects: 22864, done.
remote: Counting objects: 100% (1941/1941), done.
remote: Compressing objects: 100% (1172/1172), done.
remote: Total 22864 (delta 1017), reused 1361 (delta 743), pack-reused 20923
Receiving objects: 100% (22864/22864), 70.07 MiB | 31.03 MiB/s, done.
Resolving deltas: 100% (15856/15856), done.
/content/mmaction2
Obtaining file:///content/mmaction2
  Preparing metadata (setup.py) ... done
  Running setup.py develop for mmaction2


In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMAction2 installation
import mmaction
print(mmaction.__version__)

# Check MMCV installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

# Check MMEngine installation
from mmengine.utils.dl_utils import collect_env
print(collect_env())

2.1.0+cu118 True
1.2.0
11.8
GCC 9.3
OrderedDict([('sys.platform', 'linux'), ('Python', '3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]'), ('CUDA available', True), ('numpy_random_seed', 2147483648), ('GPU 0', 'Tesla T4'), ('CUDA_HOME', '/usr/local/cuda'), ('NVCC', 'Cuda compilation tools, release 11.8, V11.8.89'), ('GCC', 'x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0'), ('PyTorch', '2.1.0+cu118'), ('PyTorch compiling details', 'PyTorch built with:\n  - GCC 9.3\n  - C++ Version: 201703\n  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications\n  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)\n  - OpenMP 201511 (a.k.a. OpenMP 4.5)\n  - LAPACK is enabled (usually provided by MKL)\n  - NNPACK is enabled\n  - CPU capability usage: AVX512\n  - CUDA Runtime 11.8\n  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=comput

## 0. mmaction2(framework)에 맞게 파일 배치 변경

해당 구간은 로컬에서 돌린 다음 구글드라이브로 옮겼습니다.

In [ ]:
import os
path_dir = 'train/0/'
train_0_list = os.listdir(path_dir)

path_dir = 'train/1/'
train_1_list = os.listdir(path_dir)

path_dir = 'test'
test_list = os.listdir(path_dir)

FileNotFoundError: ignored

In [ ]:
df = pd.DataFrame(columns=['filename','label'])
for i in range(len(train_0_list)):
    tmp = pd.DataFrame({'filename':[train_0_list[i]], 'label':[0]})
    df = pd.concat([df, tmp], axis=0)


for i in range(len(train_1_list)):
    tmp = pd.DataFrame({'filename':[train_1_list[i]], 'label':[1]})
    df = pd.concat([df, tmp], axis=0)

df = df.sample(frac=1, random_state=0).reset_index(drop=True)

In [ ]:
train_df = df[:-int(len(df)/10)]
val_df = df[-int(len(df)/10):]

In [ ]:
len(df), len(train_df), len(val_df)

(961, 865, 96)

In [ ]:
len(train_df[train_df['label'] == 1])

48

In [ ]:
len(train_df[train_df['label'] == 0])

817

In [ ]:
len(val_df[val_df['label'] == 1])

6

In [ ]:
len(val_df[val_df['label'] == 0])

90

In [ ]:
!mkdir mmaction_data
!mkdir mmaction_data\train

In [ ]:
for name in train_df['filename'].to_list():
    if name[0] == 'f':
        os.replace('train/0/' + name, 'mmaction_data/train/' + name)
    else:
        os.replace('train/1/' + name, 'mmaction_data/train/' + name)
train_df.to_csv('mmaction_data/lumanlab_train.txt', header=False, index=False, sep=' ')


for name in val_df['filename'].to_list():
    if name[0] == 'f':
        os.replace('train/0/' + name, 'mmaction_data/val/' + name)
    else:
        os.replace('train/1/' + name, 'mmaction_data/val/' + name)
val_df.to_csv('mmaction_data/lumanlab_val.txt', header=False, index=False, sep=' ')

FileNotFoundError: [WinError 2] 지정된 파일을 찾을 수 없습니다: 'train/0/f_155.mp4' -> 'mmaction_data/train/f_155.mp4'

## 1. DataAugmentation

데이터셋의 불균형이 심함.


라벨 1인 데이터들에 대해서만 data augmentation을 통해

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/mmaction_data/lumanlab_train.txt', header=None, sep=' ')

In [ ]:
import cv2
import matplotlib.pyplot as plt

def image_flip(input_name):

  video = cv2.VideoCapture(input_name)
  video_width  = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))   # float `width`
  video_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))  # float `height`
  video_fps = int(video.get(cv2.CAP_PROP_FPS))

  fourcc = cv2.VideoWriter_fourcc(*'XVID')
  out = cv2.VideoWriter(input_name[:-4]+'f'+input_name[-4:], fourcc, video_fps, (video_width,  video_height))

  index = 0
  while (True):
    ret, frame = video.read()

    index+=1
    if not ret:

        break

    flip_frame = cv2.flip(frame, 1)


    #Show the binary frames
    if ret == True:
        out.write(flip_frame)
    else:
        break
  video.release()
  out.release()

In [ ]:
def image_rotate(input_name):
  angles = [-20, -10, 10, 20]

  for i in range(4):

    video = cv2.VideoCapture(input_name)
    video_width  = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))   # float `width`
    video_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))  # float `height`
    video_fps = int(video.get(cv2.CAP_PROP_FPS))

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(input_name[:-4]+'r'+str(i)+input_name[-4:], fourcc, video_fps, (video_width,  video_height))

    index = 0
    while (True):
      ret, frame = video.read()

      index+=1

      if not ret:

          break

      M = cv2.getRotationMatrix2D((video_width/2, video_height/2), angles[i], 1.0)
      rotate_frame = cv2.warpAffine(frame, M, (video_width, video_height))


      #Show the binary frames
      if ret == True:
          out.write(rotate_frame)
      else:
          break
    video.release()
    out.release()

In [ ]:
def image_flip_rotate(input_name):
  angles = [-20, -10, 10, 20]

  for i in range(4):

    video = cv2.VideoCapture(input_name)
    video_width  = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))   # float `width`
    video_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))  # float `height`
    video_fps = int(video.get(cv2.CAP_PROP_FPS))

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(input_name[:-4]+'fr'+str(i)+input_name[-4:], fourcc, video_fps, (video_width,  video_height))
    index = 0
    while (True):
      ret, frame = video.read()

      index+=1
      if not ret:

          break

      flip_frame = cv2.flip(frame, 1)

      M = cv2.getRotationMatrix2D((video_width/2, video_height/2), angles[i], 1.0)
      flip_rotate_video = cv2.warpAffine(flip_frame, M, (video_width, video_height))


      #Show the binary frames
      if ret == True:
          out.write(flip_rotate_video)
      else:
          break
    video.release()
    out.release()

In [ ]:
def image_crop(input_name):

  video = cv2.VideoCapture(input_name)
  video_width  = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))   # float `width`
  video_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))  # float `height`
  video_fps = int(video.get(cv2.CAP_PROP_FPS))

  fourcc = cv2.VideoWriter_fourcc(*'XVID')
  out_name = ''
  if 'train' in input_name:
    out_name = input_name.replace('train', 'train_crop')
  elif 'val' in input_name:
    out_name = input_name.replace('val', 'val_crop')
  else:
    out_name = input_name.replace('test', 'test_crop')

  crop_size = int(video_width/(1280)*280)
  out = cv2.VideoWriter(out_name, fourcc, video_fps, (crop_size*2,  crop_size*2))

  index = 0
  while (True):
    ret, frame = video.read()

    index+=1
    if not ret:

        break

    cropped_frame = frame[(video_height//2-crop_size):(video_height//2+crop_size), (video_width//2-crop_size):(video_width//2+crop_size)]


    #Show the binary frames
    if ret == True:
        out.write(cropped_frame)
    else:
        break
  video.release()
  out.release()

In [ ]:
train_path = '/content/drive/MyDrive/mmaction_data/train/'


for index, row in df.iterrows():
  #print(row[0], row[1])

  if row[1] == 1:
    print(train_path + row[0])

    ## 좌우 반전
    image_flip(train_path + row[0])

    ## 회전 -20, 10, 10, 20도
    image_rotate(train_path + row[0])
    ## 좌우 반전하고 회전 4개
    image_flip_rotate(train_path + row[0])

In [ ]:
train_list = os.listdir('/content/drive/MyDrive/mmaction_data/train')

for filename in train_list:
  print(filename)
  image_crop('/content/drive/MyDrive/mmaction_data/train/'+filename)

f_380.mp4
f_386.mp4
f_385.mp4
f_383.mp4
f_384.mp4
f_390.mp4
f_389.mp4
f_387.mp4
f_394.mp4
f_392.mp4
f_393.mp4
f_398.mp4
f_395.mp4
f_396.mp4
f_397.mp4
f_402.mp4
f_401.mp4
f_400.mp4
f_399.mp4
f_403.mp4
f_405.mp4
f_406.mp4
f_404.mp4
f_408.mp4
f_410.mp4
f_407.mp4
f_409.mp4
f_413.mp4
f_412.mp4
f_411.mp4
f_417.mp4
f_415.mp4
f_416.mp4
f_419.mp4
f_418.mp4
f_420.mp4
f_422.mp4
f_423.mp4
f_427.mp4
f_426.mp4
f_428.mp4
f_425.mp4
f_430.mp4
f_429.mp4
f_432.mp4
f_431.mp4
f_433.mp4
f_436.mp4
f_434.mp4
f_438.mp4
f_440.mp4
f_439.mp4
f_437.mp4
f_442.mp4
f_443.mp4
f_441.mp4
f_446.mp4
f_444.mp4
f_445.mp4
f_447.mp4
f_448.mp4
f_449.mp4
f_451.mp4
f_450.mp4
f_454.mp4
f_452.mp4
f_453.mp4
f_457.mp4
f_458.mp4
f_456.mp4
f_455.mp4
f_460.mp4
f_462.mp4
f_461.mp4
f_463.mp4
f_465.mp4
f_464.mp4
f_468.mp4
f_467.mp4
f_469.mp4
f_466.mp4
f_473.mp4
f_470.mp4
f_471.mp4
f_472.mp4
f_474.mp4
f_476.mp4
f_477.mp4
f_475.mp4
f_480.mp4
f_481.mp4
f_479.mp4
f_478.mp4
f_484.mp4
f_482.mp4
f_483.mp4
f_485.mp4
f_486.mp4
f_487.mp4
f_489.mp4


In [ ]:
test_list = os.listdir('/content/drive/MyDrive/mmaction_data/test')
val_list = os.listdir('/content/drive/MyDrive/mmaction_data/val')

for filename in test_list:
  image_crop('/content/drive/MyDrive/mmaction_data/test/'+filename)

for filename in val_list:
  image_crop('/content/drive/MyDrive/mmaction_data/val/'+filename)

In [ ]:
import os
train_list = os.listdir('/content/drive/MyDrive/mmaction_data/train_crop')

In [ ]:
len(train_list)

1297

In [ ]:
df = pd.DataFrame(columns=['filename','label'])
for i in range(len(train_list)):
    if train_list[i][0] == 'f':
      tmp = pd.DataFrame({'filename':[train_list[i]], 'label':[0]})
      df = pd.concat([df, tmp], axis=0)
    else:
      tmp = pd.DataFrame({'filename':[train_list[i]], 'label':[1]})
      df = pd.concat([df, tmp], axis=0)

In [ ]:
df = df.sample(frac=1, random_state=0 ).reset_index(drop=True)

In [ ]:
df.to_csv('/content/drive/MyDrive/mmaction_data/lumanlab_train_crop.txt', header=False, index=False, sep=' ')

In [ ]:
!cp /content/drive/MyDrive/mmaction_data/lumanlab_val.txt /content/drive/MyDrive/mmaction_data/lumanlab_val_crop.txt

## 2. Config 수정

In [ ]:
!pip install wget

In [ ]:
import wget
import os

In [ ]:
!mkdir checkpoints
!mkdir workdir

In [ ]:
url = "https://download.openmmlab.com/mmaction/v1.0/recognition/swin/swin-tiny-p244-w877_in1k-pre_8xb8-amp-32x2x1-30e_kinetics400-rgb/swin-tiny-p244-w877_in1k-pre_8xb8-amp-32x2x1-30e_kinetics400-rgb_20220930-241016b2.pth"

In [ ]:
wget.download(url)
os.replace('swin-tiny-p244-w877_in1k-pre_8xb8-amp-32x2x1-30e_kinetics400-rgb_20220930-241016b2.pth', 'checkpoints/' + 'swin-tiny-p244-w877_in1k-pre_8xb8-amp-32x2x1-30e_kinetics400-rgb_20220930-241016b2.pth')

In [ ]:
!dir

checkpoints	   demo     MANIFEST.in		projects	 requirements.txt  tests
CITATION.cff	   docker   mmaction		README.md	 resources	   tools
configs		   docs     mmaction2.egg-info	README_zh-CN.md  setup.cfg	   workdir
dataset-index.yml  LICENSE  model-index.yml	requirements	 setup.py


In [ ]:
from mmengine import Config
cfg = Config.fromfile('configs/recognition/swin/swin-small-p244-w877_in1k-pre_8xb8-amp-32x2x1-30e_kinetics400-rgb.py')
checkpoint = 'checkpoints/swin-tiny-p244-w877_in1k-pre_8xb8-amp-32x2x1-30e_kinetics400-rgb_20220930-241016b2.pth'


In [ ]:
cfg.train_dataloader.batch_size

8

In [ ]:
from mmengine.runner import set_random_seed

# Modify dataset type and path
cfg.data_root = '/content/drive/MyDrive/mmaction_data/train_crop/'
cfg.data_root_val = '/content/drive/MyDrive/mmaction_data/val_crop/'
cfg.ann_file_train = '/content/drive/MyDrive/mmaction_data/lumanlab_train_crop.txt'
cfg.ann_file_val = '/content/drive/MyDrive/mmaction_data/lumanlab_val_crop.txt'


cfg.test_dataloader.dataset.ann_file = '/content/drive/MyDrive/mmaction_data/lumanlab_val_crop.txt'
cfg.test_dataloader.dataset.data_prefix.video = '/content/drive/MyDrive/mmaction_data/val_crop/'

cfg.train_dataloader.dataset.ann_file = '/content/drive/MyDrive/mmaction_data/lumanlab_train_crop.txt'
cfg.train_dataloader.dataset.data_prefix.video = '/content/drive/MyDrive/mmaction_data/train_crop/'

cfg.val_dataloader.dataset.ann_file = '/content/drive/MyDrive/mmaction_data/lumanlab_val_crop.txt'
cfg.val_dataloader.dataset.data_prefix.video  = '/content/drive/MyDrive/mmaction_data/val_crop/'


# Modify num classes of the model in cls_head
cfg.model.cls_head.num_classes = 2
# We can use the pre-trained TSN model
cfg.load_from = 'checkpoints/swin-tiny-p244-w877_in1k-pre_8xb8-amp-32x2x1-30e_kinetics400-rgb_20220930-241016b2.pth'

# Set up working dir to save files and logs.
cfg.work_dir = 'workdir'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.train_dataloader.batch_size = cfg.train_dataloader.batch_size//4
cfg.val_dataloader.batch_size = cfg.val_dataloader.batch_size//4
cfg.optim_wrapper.optimizer.lr = cfg.optim_wrapper.optimizer.lr/8
cfg.train_cfg.max_epochs = 10

cfg.train_dataloader.num_workers = 2
cfg.val_dataloader.num_workers = 2
cfg.test_dataloader.num_workers = 2

# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')


Config:
ann_file_test = 'data/kinetics400/kinetics400_val_list_videos.txt'
ann_file_train = '/content/drive/MyDrive/mmaction_data/lumanlab_train_crop.txt'
ann_file_val = '/content/drive/MyDrive/mmaction_data/lumanlab_val_crop.txt'
auto_scale_lr = dict(base_batch_size=64, enable=False)
data_root = '/content/drive/MyDrive/mmaction_data/train_crop/'
data_root_val = '/content/drive/MyDrive/mmaction_data/val_crop/'
dataset_type = 'VideoDataset'
default_hooks = dict(
    checkpoint=dict(
        interval=3, max_keep_ckpts=5, save_best='auto', type='CheckpointHook'),
    logger=dict(ignore_last=False, interval=100, type='LoggerHook'),
    param_scheduler=dict(type='ParamSchedulerHook'),
    runtime_info=dict(type='RuntimeInfoHook'),
    sampler_seed=dict(type='DistSamplerSeedHook'),
    sync_buffers=dict(type='SyncBuffersHook'),
    timer=dict(type='IterTimerHook'))
default_scope = 'mmaction'
env_cfg = dict(
    cudnn_benchmark=False,
    dist_cfg=dict(backend='nccl'),
    mp_cfg=dict(mp_star

In [ ]:
cfg.train_dataloader.dataset.pipeline

[{'type': 'DecordInit', 'io_backend': 'disk'},
 {'type': 'SampleFrames', 'clip_len': 32, 'frame_interval': 2, 'num_clips': 1},
 {'type': 'DecordDecode'},
 {'type': 'Resize', 'scale': (-1, 256)},
 {'type': 'RandomResizedCrop'},
 {'type': 'Resize', 'scale': (224, 224), 'keep_ratio': False},
 {'type': 'Flip', 'flip_ratio': 0.5},
 {'type': 'FormatShape', 'input_format': 'NCTHW'},
 {'type': 'PackActionInputs'}]

## 3. 모델 수행

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
import os.path as osp
import mmengine
from mmengine.runner import Runner

# Create work_dir
mmengine.mkdir_or_exist(osp.abspath(cfg.work_dir))

# build the runner from config
runner = Runner.from_cfg(cfg)

# start training
runner.train()

10/22 14:15:04 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
    CUDA available: True
    numpy_random_seed: 2094263368
    GPU 0: Tesla T4
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.1.0+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


10/22 14:15:08 - mmengine - INFO - Distributed training is not used, all SyncBatchNorm (SyncBN) layers in the model will be automatically reverted to BatchNormXd layers if they are used.
10/22 14:15:08 - mmengine - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) RuntimeInfoHook                    
(BELOW_NORMAL) LoggerHook                         
 -------------------- 
before_train:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(VERY_LOW    ) CheckpointHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(NORMAL      ) DistSamplerSeedHook                
 -------------------- 
before_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
 -------------------- 
after_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                

Recognizer3D(
  (data_preprocessor): ActionDataPreprocessor()
  (backbone): SwinTransformer3D(
    (patch_embed): PatchEmbed3D(
      (proj): Conv3d(3, 96, kernel_size=(2, 4, 4), stride=(2, 4, 4))
      (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0): BasicLayer(
        (blocks): ModuleList(
          (0): SwinTransformerBlock3D(
            (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
            (attn): WindowAttention3D(
              (qkv): Linear(in_features=96, out_features=288, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=96, out_features=96, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
              (softmax): Softmax(dim=-1)
            )
            (drop_path): Identity()
            (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
            (mlp): Mlp(
    

In [ ]:
!dir ./checkpoints/

swin-tiny-p244-w877_in1k-pre_8xb8-amp-32x2x1-30e_kinetics400-rgb_20220930-241016b2.pth


In [ ]:
!cp ./checkpoints/swin-tiny-p244-w877_in1k-pre_8xb8-amp-32x2x1-30e_kinetics400-rgb_20220930-241016b2.pth /content/drive/MyDrive/mmaction_data/checkpoints/swin-202310222128.pth

## 4. inference

In [ ]:
checkpoint = '/content/drive/MyDrive/mmaction_data/checkpoints/swin-202310222128.pth'

In [ ]:
# Use the recognizer to do inference
from operator import itemgetter
import os

test_path = '/content/drive/MyDrive/mmaction_data/test_crop/'

test_list = os.listdir(test_path)

In [ ]:
test_list[:5]

['t_34.mp4', 't_22.mp4', 't_35.mp4', 't_18.mp4', 't_33.mp4']

In [ ]:
from mmaction.apis import inference_recognizer, init_recognizer

# Initialize the recognizer
model = init_recognizer(cfg, checkpoint, device='cuda:0')

Loads checkpoint by local backend from path: /content/drive/MyDrive/mmaction_data/checkpoints/swin-202310222128.pth
The model and loaded state dict do not match exactly

size mismatch for cls_head.fc_cls.weight: copying a param with shape torch.Size([400, 768]) from checkpoint, the shape in current model is torch.Size([2, 768]).
size mismatch for cls_head.fc_cls.bias: copying a param with shape torch.Size([400]) from checkpoint, the shape in current model is torch.Size([2]).
missing keys in source state_dict: backbone.layers.2.blocks.6.norm1.weight, backbone.layers.2.blocks.6.norm1.bias, backbone.layers.2.blocks.6.attn.relative_position_bias_table, backbone.layers.2.blocks.6.attn.relative_position_index, backbone.layers.2.blocks.6.attn.qkv.weight, backbone.layers.2.blocks.6.attn.qkv.bias, backbone.layers.2.blocks.6.attn.proj.weight, backbone.layers.2.blocks.6.attn.proj.bias, backbone.layers.2.blocks.6.norm2.weight, backbone.layers.2.blocks.6.norm2.bias, backbone.layers.2.blocks.6.mlp.f

In [ ]:
test_df = pd.DataFrame(columns=['video_name','label'])


for filename in test_list:
  video = test_path + filename
  print(video)
  results = inference_recognizer(model, video)

  tmp = pd.DataFrame({'video_name':[filename], 'label':[results.pred_label.tolist()[0]]})
  test_df = pd.concat([test_df, tmp], axis=0)


test_df.to_csv('/content/drive/MyDrive/mmaction_data/prediction_crop.csv', index=False, sep=',')

/content/drive/MyDrive/mmaction_data/test_crop/t_34.mp4
/content/drive/MyDrive/mmaction_data/test_crop/t_22.mp4
/content/drive/MyDrive/mmaction_data/test_crop/t_35.mp4
/content/drive/MyDrive/mmaction_data/test_crop/t_18.mp4
/content/drive/MyDrive/mmaction_data/test_crop/t_33.mp4
/content/drive/MyDrive/mmaction_data/test_crop/t_21.mp4
/content/drive/MyDrive/mmaction_data/test_crop/t_31.mp4
/content/drive/MyDrive/mmaction_data/test_crop/t_30.mp4
/content/drive/MyDrive/mmaction_data/test_crop/t_29.mp4
/content/drive/MyDrive/mmaction_data/test_crop/t_32.mp4
/content/drive/MyDrive/mmaction_data/test_crop/t_27.mp4
/content/drive/MyDrive/mmaction_data/test_crop/t_20.mp4
/content/drive/MyDrive/mmaction_data/test_crop/t_28.mp4
/content/drive/MyDrive/mmaction_data/test_crop/t_16.mp4
/content/drive/MyDrive/mmaction_data/test_crop/t_26.mp4
/content/drive/MyDrive/mmaction_data/test_crop/t_24.mp4
/content/drive/MyDrive/mmaction_data/test_crop/t_25.mp4
/content/drive/MyDrive/mmaction_data/test_crop/t

In [ ]:
!pip freeze > /content/mmaction2/requirements.txt